In [ ]:
# testing the SQL connection
from src.llm_inference import TextInference
from src.database_connector import DatabaseAgent
from src.prompt_formatter import PromptFormatterV1
from src.utils import SQLExtractor, get_data_from_query

In [ ]:
username = 'root'   
password = 'core'  
hostname = 'localhost'
database_name = 'classicmodels'  

sql_agent = DatabaseAgent(host_name=hostname,port=3306,username=username,database_name=database_name,password=password, database_type="MySQL")
tables = sql_agent.grab_table_schema(sql_agent.grab_table_names())
prompt_format = PromptFormatterV1(tables=tables, db_type="MySQL")

In [ ]:
prompt =  prompt_format(question="Names of customers having more than 10K in a single basket, along with their purchase amount")
inference = TextInference(model_name="NumbersStation/nsql-350M")
output = inference.generate_text(prompt, max_length=1024)
extractor = SQLExtractor(text=output)
result = extractor.extract_select_commands()[-1]
print(result)

In [ ]:
get_data_from_query(query=result,db_url=sql_agent.conn_str)

In [1]:
# testing the SQL connection
from src.llm_inference import TextInference
from src.database_connector import DatabaseAgent
from src.prompt_formatter import PromptFormatterV1
from src.utils import SQLExtractor, get_data_from_query

/home/rjn/miniconda3/envs/ttb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
username = 'postgres'   
password = ''  
hostname = 'localhost'
database_name = 'companydb'  

sql_agent = DatabaseAgent(host_name=hostname,port=5432,username=username,database_name=database_name,password=password, database_type="PostgreSQL")

Connection to MySQL database successful!


In [3]:
tables = sql_agent.grab_table_schema(sql_agent.grab_table_names())
prompt_format = PromptFormatterV1(tables=tables, db_type="MySQL")

In [4]:
prompt =  prompt_format(question="Give me name and salary of employees whose salary is greater than 50000.")
inference = TextInference(model_name="NumbersStation/nsql-350M")
output = inference.generate_text(prompt, max_length=1024)
extractor = SQLExtractor(text=output)
result = extractor.extract_select_commands()[-1]
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


SELECT name, salary FROM employees WHERE salary > 50000;


In [5]:
get_data_from_query(result,sql_agent.conn_str)

/home/rjn/Documents/GitHub/TalkToDatabase/src/utils.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(str(query), raw_conn, params=params)


,name,salary
0,Alice,80000
1,Bob,60000
2,Carol,55000
3,Dave,62000
